In [67]:
from bs4 import BeautifulSoup
import requests
import json
import urllib

In [68]:
def salva(text):
    with open('text.html', 'w') as file:
        file.write(r.text.encode('utf-8'))

In [69]:
"""
obtem os deputados da requisicao inicial
"""
session = requests.Session()
deputados = []

url_verbas = "http://www.al.ma.leg.br/portal/grid_tab_usuarios/grid_tab_usuarios.php"

session.get(url_verbas)

data = {
    'nmgp_parms': 'parm1?#??@?',
    'nmgp_opcao': 'alterar'
}

r = session.post(url_verbas, data=data)

soup = BeautifulSoup(r.text, "html.parser")
table = soup.find('table', class_="scGridTabela")

for a in table.find_all('a'):
    dep = a.text
    deputados.append(dep)
    
deputados

[u'ALEXANDRE VICENTE DE PAULA ALMEIDA',
 u'ANA DE NAZAR\xc9 PEREIRA SILVA MACEDO MENDON\xc7A',
 u'ANDREA TROV\xc3O MURAD BARROS',
 u'ANT\xd4NIO PEREIRA FILHO',
 u'CARLOS WELLINGTON DE CASTRO BEZERRA',
 u'C\xc9SAR HENRIQUE SANTOS PIRES',
 u'DOMINGOS ERINALDO SOUSA SERRA',
 u'EDIL\xc1ZIO GOMES DA SILVA J\xdaNIOR',
 u'EDIVALDO DE HOLANDA BRAGA',
 u'EDSON CUNHA DE ARA\xdaJO',
 u'EDUARDO SALIM BRAIDE',
 u'F\xc1BIO HENRIQUE DIAS DE MACEDO',
 u'F\xc1BIO HENRIQUE RAMOS BRAGA',
 u'FRANCISCA FERREIRA',
 u'FRANCISCO DE SOUSA DIAS NETO']

In [70]:
"""
Obtem os deputados que sao retornados quando muda a pagina
"""
inc = 15
parm = 1
while True:
    parm = parm + inc
    print parm
    data = {
        'parm': str(parm),
        'nmgp_opcao': 'ajax_navigate',
        'opc': 'rec'
    }

    r = session.post(url_verbas, data=data)

    ajax_response = json.loads(r.text)

    ajax_html = ajax_response['setValue'][1]['value']

    soup = BeautifulSoup(ajax_html, "html.parser")
    table = soup.find('table', class_="scGridTabela")

    if not table: break;

    for a in table.find_all('a'):
        dep = a.text
        deputados.append(dep)
    
print deputados
print 'total:', len(deputados)

16
31
46
61
[u'ALEXANDRE VICENTE DE PAULA ALMEIDA', u'ANA DE NAZAR\xc9 PEREIRA SILVA MACEDO MENDON\xc7A', u'ANDREA TROV\xc3O MURAD BARROS', u'ANT\xd4NIO PEREIRA FILHO', u'CARLOS WELLINGTON DE CASTRO BEZERRA', u'C\xc9SAR HENRIQUE SANTOS PIRES', u'DOMINGOS ERINALDO SOUSA SERRA', u'EDIL\xc1ZIO GOMES DA SILVA J\xdaNIOR', u'EDIVALDO DE HOLANDA BRAGA', u'EDSON CUNHA DE ARA\xdaJO', u'EDUARDO SALIM BRAIDE', u'F\xc1BIO HENRIQUE DIAS DE MACEDO', u'F\xc1BIO HENRIQUE RAMOS BRAGA', u'FRANCISCA FERREIRA', u'FRANCISCO DE SOUSA DIAS NETO', u'GLALBERT NASCIMENTO CUTRIM', u'HEMET\xc9RIO WEBA FILHO', u'HUMBERTO IVAR DE ARA\xdaJO COUTINHO', u'JESUINO CORDEIRO MENDES JUNIOR', u'JOAQUIM ELIAS NAGIB PINTO HAICKEL', u'JOS\xc9 ADRIANO CORDEIRO SARNEY', u'JOS\xc9 ARIMATEIA LIMA NETO EVANGELISTA', u'JOS\xc9 CARLOS NOBRE MONTEIRO', u'JOS\xc9 IN\xc1CIO SODR\xc9 RODRIGUES', u'JOS\xc9 MAX PEREIRA BARROS', u'JOS\xc9 ROBERTO COSTA SANTOS', u'JOSIMAR CUNHA RODRIGUES', u'LEOARREN T\xdaLIO DE SOUSA CUNHA ', u'LEVI PONTES

In [144]:
"""Obter as despesas de cada parlamentar"""
def valores_indenizacao(nome_dep, competencia):
    
    frame = []
    
    url_control = 'http://www.al.ma.leg.br/portal/control_verbas/control_verbas.php'

    data = {
        'nomefun': nome_dep[:20].encode('latin1'), #site limita a 20 caracteres
        'nmgp_opcao': 'alterar',
        'competencia': competencia

    }

    r = session.post(url_control, data=data)

    soup = BeautifulSoup(r.text, "html.parser")
    nmgp_parms = soup.find('input', {'name': 'nmgp_parms'})

    print nmgp_parms['value'].encode('utf-8')

    data = {
        'nmgp_parms': nmgp_parms['value'],
        'nmgp_url_saida': '/portal/control_verbas/control_verbas.php',
    }

    r = session.post(url_verbas, data=data)

    #print r.text
    #salva(r.text)

    soup = BeautifulSoup(r.text, "html.parser")
    link = soup.find('a', class_="scGridFieldOddLink css_nomusu_grid_line")
    link_id = link['href'].split()[2]

    print link_id[1:-2]

    url_tab_verbas = "http://www.al.ma.leg.br/portal/grid_tab_verbas/grid_tab_verbas.php"
    data = {
        'nmgp_parms': link_id[1:-2]
    }

    r = session.post(url_tab_verbas, data=data)

    soup = BeautifulSoup(r.text, "html.parser")
    table = soup.find('table', class_= 'scGridTabela')

    linhas = table.findChildren('tr')[3:]
    total = linhas[-1:]

    colunas = linhas[0].findChildren('td')


    for linha in linhas[:-1]:
        dados = {'Nome': nome_dep, 'Competência': competencia}
        
        colunas = linha.findChildren('td')
        dados['codigo'] = colunas[0].find('span').text
        dados['descrição'] = colunas[1].find('span').text
        dados['valor'] = colunas[2].find('span').text
        
        frame.append(dados)
        
    #print total
    return frame
    

In [148]:
print valores_indenizacao(deputados[1], '082017')

parm1?#?ANA DE NAZAR� PEREIR?@?
@SC_par@7023@SC_par@grid_tab_usuarios@SC_par@7408b486c2ca0aeba516a5080dd605c9
[{'valor': u'12.576,63', 'codigo': u'09', 'descri\xc3\xa7\xc3\xa3o': u'Combust\xedveis e lubrificantes', 'Compet\xc3\xaancia': '082017', 'Nome': u'ANA DE NAZAR\xc9 PEREIRA SILVA MACEDO MENDON\xc7A'}, {'valor': u'7.000,00', 'codigo': u'11', 'descri\xc3\xa7\xc3\xa3o': u'Contrata\xe7\xe3o de assessorias, consultorias e trabalhos t\xe9cnicos ', 'Compet\xc3\xaancia': '082017', 'Nome': u'ANA DE NAZAR\xc9 PEREIRA SILVA MACEDO MENDON\xc7A'}, {'valor': u'9.800,00', 'codigo': u'17', 'descri\xc3\xa7\xc3\xa3o': u'Di\xe1ria de servidor do gabinete', 'Compet\xc3\xaancia': '082017', 'Nome': u'ANA DE NAZAR\xc9 PEREIRA SILVA MACEDO MENDON\xc7A'}, {'valor': u'8.153,62', 'codigo': u'06', 'descri\xc3\xa7\xc3\xa3o': u'Fornecimento de alimenta\xe7\xe3o ao parlamentar e assessores', 'Compet\xc3\xaancia': '082017', 'Nome': u'ANA DE NAZAR\xc9 PEREIRA SILVA MACEDO MENDON\xc7A'}, {'valor': u'574,00', 'co

In [137]:
valores_indenizacao(deputados[1], '032017')

parm1?#?ANA DE NAZAR� PEREIR?@?
@SC_par@7778@SC_par@grid_tab_usuarios@SC_par@7408b486c2ca0aeba516a5080dd605c9
código 09 descrição Combustíveis e lubrificantes valor 11.768,85
código 11 descrição Contratação de assessorias, consultorias e trabalhos técnicos  valor 6.000,00
código 17 descrição Diária de servidor do gabinete valor 13.645,00
código 06 descrição Fornecimento de alimentação ao parlamentar e assessores valor 5.937,63
código 16 descrição Manutenção, peças e acessórios de veículos valor 1.598,31
código 13 descrição Material de expediente e suprimentos de informática valor 2.340,98
código 02 descrição Telefonia valor 1.501,67
[<tr class="scGridSubtotal">\n<td align="left" class="scGridSubtotalFont" colspan="2" nowrap="">O valor ressarcido \xe9 R$ 41.779,83 - Total gasto no m\xeas: </td>\n<td align="" class="scGridSubtotalFont css_valor_sub_tot" nowrap="">42.792,44</td>\n</tr>]
